In [19]:
#setup 3D Cone
%run -i program/billiard_defs.py
%run -i program/analysis.py

dim = 3
max_steps = 50
pw_collision_law = 'no_slip'
# pw_collision_law = 'specular'

cone_vertex = np.array([0,0,0])
cone_axis = munit(np.array([0,0,1]))
cone_height = 100
cone_vertex_angle = np.pi/6
r = cone_height * np.tan(cone_vertex_angle)

wall = [ConeWall(name = 'cone', pos=cone_vertex, axis=cone_axis, height=cone_height, vertex_angle=cone_vertex_angle, collision_law=pw_collision_law)
        #,FlatWall(name='top', pos=cone_vertex+cone_axis*cone_height, normal=-1*cone_axis, half_length=r, collision_law=pw_collision_law),
       ]
lab_frame = wall[0].basis.copy()

bounding_box = np.array([[-r,r]
                         ,[-r,r]
                         ,[0,wall[0].height]
                        ])

# Used for visualization later
surface_data_dict = []
for w in wall:
    w.parametrize()
    data_dict = {
        'name': w.name,
        'type': 'surface',
        'opacity': 0.4,
        'colorscale': [[0, 'rgb(0,0,255)'], [1, 'rgb(0,0,255)']],
        'showscale': False,
        'x': w.param[0],
        'y': w.param[1],
        'z': w.param[2]
    }
    surface_data_dict.append(data_dict)

In [20]:
# Setup particles

num_part = 1
mass = 1
gamma = 'uniform'  #parameter btw 0 mass at center) and sqrt(2/dim) (mass on outside shell)
radius = 1
temp = 10
pp_collision_law = 'no_slip'
# pp_collision_law = 'specular'

init_pos = [[0,0,10]]
init_vel = [[1,0,0]]


In [21]:
%run -i program/billiard_defs.py
%run -i program/analysis.py

max_steps = 40
max_distort = 1000  # Used after experiment to determine whether to interpolate between collisions
# If so, how much distortion to allow.  Must be 0-100, smaller <-> more frames and smoother, but longer 
# processing and larger files.  Anything other than 0-100 will turn off smoothing

try:
    step = 'init'
    part = Particles(wall, num=num_part, mass=mass, gamma=gamma
                 ,radius=radius, temp=temp, collision_law=pp_collision_law
                 ,pos=init_pos  # comment this line out for random init pos
                 ,vel=init_vel  # comment this line out for random init vel
                 #,spin=init_spin
                )
    init_spin = np.array([0, 0, wall[0].sinva * part.vel[0,0] / (part.radius[0] * part.gamma[0]**2)])
    part.spin[0] = part.v_to_m(init_spin)
    print(part.pos)
    print(part.vel)
    print(part.spin)
    init(wall, part)
    for step in range(max_steps):
        do_the_evolution()
except:
    error_report()

re_t, re_pos, re_vel, re_rot, re_spin = smoother(max_distort)
S, P, D, M = re_pos.shape

print("Finished it all.  There are {} frames in this animation of {} particles.".format(S, P))

[[  0.   0.  10.]]
[[ 1.  0.  0.]]
[[[ 0.   -1.25  0.  ]
  [ 1.25  0.   -0.  ]
  [ 0.    0.    0.  ]]]
Finished it all.  There are 41 frames in this animation of 1 particles.


In [22]:
interactive_plotly()

using newer
